# Laboratorio 5: Introducción a Redes Neuronales Recurrentes.

##### <strong>RNN y PyTorch </strong>

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

In [ ]:
!pip uninstall torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.12/dist-packages/functorch/*
    /usr/local/lib/python3.12/dist-packages/torch-2.8.0+cu126.dist-info/*
    /usr/local/lib/python3.12/dist-packages/torch/*
    /usr/local/lib/python3.12/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Would remove:
    /usr/local/lib/python3.12/dist-packages/torchvision-0.23.0+cu126.dist-info/*
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libcudart.45e7f3ed.so.12
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libjpeg.bd6b9199.so.8
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libnvjpeg.e5f20359.so.12
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libpng16.0481ee11.so.16
    /usr/local/l

In [ ]:
!pip uninstall scipy
!pip install scipy==1.11.4

In [ ]:
!pip install torchtext

In [ ]:
!pip install scikit-plot

In [ ]:
import csv
import pandas as pd
from google.colab import files

import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import scikitplot as skplt
import matplotlib.pyplot as plt
import numpy as np


## Repaso: Redes Neuronales Recurrentes para Clasificación de texto

Los tipos de redes neuronales como las redes fully connected  o las convolucionales son buenas para identificar patrones en los datos, pero no tienen memoria. Tratan cada ejemplo de datos y partes del ejemplo como independientes entre sí. No pueden mantener ningún estado/memoria sobre ejemplos previamente vistos. Este tipo de comportamiento es bueno siempre y cuando cada ejemplo, como las imágenes, sea independiente de los demás. Pero hay situaciones en las que recordar información de estado sobre ejemplos previamente vistos puede ayudar a obtener mejores resultados. Digamos, por ejemplo, en la tarea de procesamiento de lenguaje natural de generación de texto, si nuestra red puede recordar alguna información de estado sobre las palabras vistas, entonces puede ayudar a recordar el estado y generar nuevas palabras mejores ya que ahora conoce el contexto de la oración. Este tipo de enfoque también puede ayudar con datos de series temporales, donde la nueva predicción generalmente depende de los últimos ejemplos de texto.

Para resolver el problema de mantener la memoria, se introdujeron las redes neuronales recurrentes (RNN). Las redes neuronales recurrentes mantienen el estado de los ejemplos de datos y lo utilizan para mejorar los resultados. Si el lector está interesado en aprender sobre el funcionamiento interno de las RNN, recomendamos este blog que lo cubre en detalle.

Como parte de este tutorial, vamos a diseñar RNNs simples usando PyTorch para resolver tareas de clasificación de texto. Probaremos diferentes enfoques para usar RNNs en la clasificación de documentos de texto. Utilizaremos el enfoque de incrustación de palabras para vectorizar palabras en vectores de valores reales antes de proporcionárselos a las RNNs. El objetivo principal del tutorial es iniciar a las personas en el uso de RNNs para tareas de clasificación de texto.

<img src="https://ashutoshtripathicom.files.wordpress.com/2021/06/rnn-vs-lstm.png">

<center>Arquitectura de una Red Recurrente y una LSTM.</center>





### Cargar el dataset

En esta sección, hemos cargaremos el conjunto de datos `AG_NEWS` para crear un vocabulario utilizando tokens generados a partir de ejemplos de texto del conjunto de datos. Posteriormente, el vocabulario se utilizará para mapear tokens a índices que se utilizarán para identificarlos. Estos índices generados para tokens de ejemplos de texto se darán como entrada a las redes neuronales para clasificar documentos de texto.

Para esto debemos cargar los archivos de entrenamientos y testing:

In [ ]:
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/tutoriales/data/ag_news/train.csv
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/tutoriales/data/ag_news/test.csv

El `AG_NEWS` dataset, es un conjunto de datos que clasifica diferentes artículos de noticias de acuerdos a su contenidos, por lo que presenta la siguientes columnas:

- El índice de la clase a la pertence el artículo.
- El título del artículo.
- La Descripción del artículo

En el siguiente [link](https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset) es posible encontrar más información del dataset.

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

Para que `PyTorch` sea capaz tomar la información del dataset, es necesario cargarlo a través de un generador, para eso creamos la siguiente función que extrae la etiqueta y el contenido de la noticias.

In [ ]:
def load_dataset(dataset):
  with open(dataset, encoding='utf-8') as dataset_file:
    reader = csv.reader(dataset_file)
    next(reader)
    for row in reader:
      yield int(row[0]), f'{row[1]} {row[2]}'

In [ ]:
next(load_dataset('test.csv'))

Luego, ejecutamos la función tanto los datasets de `train` y `test`:

In [ ]:
train_dataset = load_dataset('train.csv')
test_dataset = load_dataset('test.csv')

### Crear vocabulario

Una vez que hemos cargado el dataset, es necesario crear nuestro vocabulario a partir de los tokens que componen nuestro dataset. Para esto, se deben seguir los siguientes pasos:

- Es necesario tokenizar el dataset.
- A partir del el dataset tokenizado, se crea el vocabulario.

Para crear el vocabulario usaremos la API de `torchtext`, una librería de `PyTorch`, que contiene una API para trabajar usando métodos de deep learning para NLP.

In [ ]:
tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

Internamente, la estructura del vocabulario, le asigna un índice a token dentro del vocaluario. Como se ve en el siguiente anterior. Con esto es posible obtener todos los indice de una oración dentro del corpus de texto.

In [ ]:
tokens = tokenizer("Hello how are you?, Welcome to CoderzColumn!!")
indexes = vocab(tokens)

tokens, indexes

### Cargar el `dataloader`.

Para crear el dataloader, debemos cargar los datos a una instancia de la clase `Dataset` de `PyTorch`, sin embargo podemos usar la función `to_map_style_dataset` para no implementar la clase completa. Ojo que el uso de la función depende caso, a veces es inevitable tener que implementar su propio wrapper.

In [ ]:
train_dataset = load_dataset('train.csv')
test_dataset = load_dataset('test.csv')
train_dataset, test_dataset  = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

target_classes = ["World", "Sports", "Business", "Sci/Tech"]

max_words = 25


Dado que iremos transformando el dataset a medida que extrayendo cada `batch` de texto, es necesario definir una función que haga esta conversión.

In [ ]:
def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1 ## We have deducted 1 from target names to get them in range [0,1,2,3] from [1,2,3,4]

Luego, definimos nuestra instancia del `DataLoader`, este objeto será aquel que irá iterando y extrayendo cada `batch` del dataset.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)

### Definición Red Neuronal Recurrente

Previamente, a la definición de la Red Neuronal debemos definir los hiperparametros de está. No es una obligación, pero siempre es bueno hacerlo antes, para entender las dimensiones de nuestra red.

In [ ]:

embed_len = 50
hidden_dim = 50
n_layers=1

Para definir una Red Neuronal en `PyTorch`, debemos extender la clase `torch.Module`, en este caso `torch` fue renombrado como `nn`. Dentro de la constructor de la clase se definen todas las capas de la que nuestra red neuronal, en el caso de la tarea que buscamos resolver, se definen las siguientes:

- Una capa de `Embedding`, para encodar cada una de las palabras del vocabulario a un vector denso.
- Una capa de `RNN`, que implementa la arquitectura de `RNN`, por lo que no es necesario hacerlo a mano. Esta red agrega la secuencia de temporalidad de cada de las de palabras, considerando el contexto de la frase.
- Una capa `Linear`, esta capa se encarga de resolver la tarea de clasificación que buscamos.

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings, torch.randn(n_layers, len(X_batch), hidden_dim))
        return self.linear(output[:,-1])

Definimos el clasificador.

In [ ]:
rnn_classifier = RNNClassifier()

rnn_classifier

Para explorar las dimensiones de nuestras capas, podemos ejecutar:

In [ ]:
for layer in rnn_classifier.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print()

### Funciones de entrenamiento y Evaluación

Para extraer los valores de accuracy es necesario calcularlos a medida que se entrena el `batch`, para esto definimos una función que se encarga de entrenar en los modelo en cada iteración, guardando la loss y el accuracy en listas estandar de Python, para esto se implementan dos funciones:

- `CalValLossAndAccuracy`: calcula la loss de la función de pérdida y guarda el accuracy que servirá para evaluar el modelo.
- `TrainModel`: gestiona el entrenamiento del modelo en base a la cantidad de épocas escogidas por el usuario.

In [ ]:


def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

### Entrenamiento del modelo.

Ejecutamos el entramiento del modelo, esto tardará algunos minutos:

In [ ]:


epochs = 15
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

### Realizar prediciones de la Red entrenada.

Para realizar las predicciones del modelo, es necesario evaluar su performance en el conjunto de entrenamiento. Para esto definimos la función `MakePredictions`, que calcula la loss del conjunto de testing y guarda las predicciones en una lista estandar de Python.

In [ ]:
def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()

Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

Utlizamos las funciones de `scikit-learn` para generar el reporte de clasificación.

In [ ]:


print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

In [ ]:

skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_actual], [target_classes[i] for i in Y_preds],
                                    normalize=True,
                                    title="Confusion Matrix",
                                    cmap="Purples",
                                    hide_zeros=True,
                                    figsize=(5,5)
                                    );
plt.xticks(rotation=90);

### Stackear muchas redes recurrentes.

Este código es similar al anterior, la diferencia es que variamos la cantidad de RNN utilizadas proponiendo una nueva arquitectura que podría mejorar o empeorar la performance del modelo.

In [ ]:
embed_len = 50
hidden_dim1 = 50
hidden_dim2 = 60
hidden_dim3 = 75
n_layers=1

class StackingRNNClassifier(nn.Module):
    def __init__(self):
        super(StackingRNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn1 = nn.RNN(input_size=embed_len, hidden_size=hidden_dim1, num_layers=1, batch_first=True)
        self.rnn2 = nn.RNN(input_size=hidden_dim1, hidden_size=hidden_dim2, num_layers=1, batch_first=True)
        self.rnn3 = nn.RNN(input_size=hidden_dim2, hidden_size=hidden_dim3, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_dim3, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn1(embeddings, torch.randn(n_layers, len(X_batch), hidden_dim1))
        output, hidden = self.rnn2(output, torch.randn(n_layers, len(X_batch), hidden_dim2))
        output, hidden = self.rnn3(output, torch.randn(n_layers, len(X_batch), hidden_dim3))
        return self.linear(output[:,-1])

In [ ]:
rnn_classifier = StackingRNNClassifier()

rnn_classifier

In [ ]:
from torch.optim import Adam

epochs = 15
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = StackingRNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

In [ ]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

In [ ]:
import numpy as np

skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_actual], [target_classes[i] for i in Y_preds],
                                    normalize=True,
                                    title="Confusion Matrix",
                                    cmap="Purples",
                                    hide_zeros=True,
                                    figsize=(5,5)
                                    );
plt.xticks(rotation=90);

## P1: Cambio de arquitecturas

Postule al menos 2 hipótesis sobre como podrían afectar a los resultados cambios en la red con distintos arquitecturas RNN. Los cambios a probar son:

- Pruebe cambiar la capa de RNN en el modelo original con otras arquitecturas como GRU  y LSTM. ¿Hay diferencias en los resultados? ¿A qué podría deberse?

- Pruebe cambiar el número de capas con distintas arquitecturas RNN. Nuevamente, ¿observa cambios?



### Hipótesis 1

### Hipótesis 2

## P2: Más experimentos

Pruebe a utilizar una red RNN para resolver algún otro problema de clasificación. Por ejemplo, podría usar el corpora que vimos en tutoriales anteriores relacionado a predecir si un texto corresponde a `dental` o `no_dental`.

Recuerde realizar todos los cambios correspondientes en la red, incluidos por ejemplo:

- Cambiar el output (por ejemplo en caso de que cambie el número de clases a predecir)
- Cambiar la función de activación y la función de loss (por ejemplo para problemas de clasificación binaria)

## P3 (Opcional): Flujo de dato en la red

Pruebe pasar un dato programando "manualmente" su flujo a través de la red original (el flujo que sigue la función `predict`). Seleccione un dato al azar del corpus y realice las operaciones matemáticas correspondientes para predecir su clase correspondiente.

Puede utilizar las librerías que necesite para realizar dichas operaciones.

¡Hacer este ejercicio le puede ayudar a tener una mejor comprensión de las operaciones aplicadas a cada dato!